<h3>Start by creating a new Notebook for this assignment.</h3>

<h3>Toronto Neighborhoods</h3>
<h3>Importing python libraries</h3>

In [10]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<h3>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M </h3>

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
#print(result.status_code)
#print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


<h3>Create a dataframe and the dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</h3>

In [12]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [13]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


<h4>custom groupby to merge Neighborhoods</h4>
<h4>groupby PostalCode, keep the first Borough and join() Neighborhoods</h4>


In [14]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

<h4>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</h4>

In [15]:
df.shape

(103, 3)

In [16]:
print("Number of rows and columns in the dataframe:", df.shape)

Number of rows and columns in the dataframe: (103, 3)


<h4>Read geo data from a CSV</h4>

In [17]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [18]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [19]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [21]:
df2.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


<h4>Explore and cluster the neighborhoods in Toronto</h4>

In [22]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [23]:
CLIENT_ID = 'UVU5KIDBNPEWAY34SNNAVXUIYH4GIYQFIMJQQSGKPI1FFCDA' # Foursquare ID
CLIENT_SECRET = 'U50CCENVH3AXVPXZ5IIRV0JEXWUSV0HFB0EQIB4AURRQFUSK' # Foursquare Secret
VERSION = '20180405' # API version

In [24]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UVU5KIDBNPEWAY34SNNAVXUIYH4GIYQFIMJQQSGKPI1FFCDA&client_secret=U50CCENVH3AXVPXZ5IIRV0JEXWUSV0HFB0EQIB4AURRQFUSK&v=20180405&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
#results

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']

In [32]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.beenHere.count,venue.beenHere.lastCheckinExpiredAt,venue.beenHere.marked,venue.beenHere.unconfirmedCount,venue.categories,venue.hereNow.count,venue.hereNow.groups,...,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.stats.checkinsCount,venue.stats.tipCount,venue.stats.usersCount,venue.stats.visitsCount,venue.verified
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-0,0,0,False,0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[],...,M9V 1B4,ON,Sheriff's No Frills,0,[],0,0,0,0,True
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-1,0,0,False,0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[],...,M9V 3Y5,ON,Subway,0,[],0,0,0,0,False
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-2,0,0,False,0,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[],...,M9V 1B4,ON,Shoppers Drug Mart,0,[],0,0,0,0,True
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-3,0,0,False,0,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[],...,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[],0,0,0,0,False
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-4,0,0,False,0,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[],...,M9V 1B4,ON,The Beer Store,0,[],0,0,0,0,False
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-5,0,0,False,0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],...,M9V 1B4,ON,Pizza Pizza,0,[],0,0,0,0,True
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-6,0,0,False,0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[],...,NaN,ON,Sunny Foodmart,0,[],0,0,0,0,False
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-7,0,0,False,0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[],...,M9V 1B4,ON,McDonald's,0,[],0,0,0,0,False
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd9d00734bb8cfa6576babf-8,0,0,False,0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],...,M9V 0A1,ON,Tim Hortons,0,[],0,0,0,0,False


In [33]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
1,Subway,Sandwich Place,43.742421,-79.589471
2,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347
3,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
4,The Beer Store,Beer Store,43.741694,-79.584373


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [37]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [38]:
print(venues.shape)
venues.head()

(2246, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [39]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1294,1294,1294,1294,1294,1294
East Toronto,117,117,117,117,117,117
East York,74,74,74,74,74,74
Etobicoke,73,73,73,73,73,73
Mississauga,11,11,11,11,11,11
North York,239,239,239,239,239,239
Queen's Park,40,40,40,40,40,40
Scarborough,85,85,85,85,85,85


In [40]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 273 unique categories.


In [41]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
onehot.shape

(2246, 273)

In [43]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008929,0.000000,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000000,0.000773,0.000773,0.000773,0.000773,0.000773,0.001546,0.002318,0.001546,...,0.000000,0.012365,0.001546,0.000000,0.004637,0.000000,0.006182,0.000773,0.000000,0.003091
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013514,0.000000,0.013514,0.000000,0.000000,0.000000,0.013514
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004184,0.000000,0.000000,0.004184,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004184,0.004184,0.008368,0.000000,0.000000,0.004184,0.008368,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011173,0.000000,0.000000,0.011173,0.000000,0.005587,0.000000,0.000000,0.011173


In [44]:
grouped.shape

(11, 273)

In [45]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              venue  freq
0       Coffee Shop  0.08
1    Sandwich Place  0.07
2  Sushi Restaurant  0.04
3    Clothing Store  0.04
4              Café  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2        Hotel  0.03
3       Bakery  0.03
4   Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4                Café  0.04


----East York----
            venue  freq
0     Coffee Shop  0.07
1            Park  0.05
2    Burger Joint  0.04
3  Sandwich Place  0.04
4        Pharmacy  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.10
1        Sandwich Place  0.07
2              Pharmacy  0.05
3         Grocery Store  0.04
4  Fast Food Restaurant  0.04


----Mississauga----
                       venue  freq
0                      Hotel  0.18
1                Cof

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Clothing Store,Pizza Place,Café,Park,Sushi Restaurant,Dessert Shop,Gym,Burger Joint
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Park,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Bookstore,Pub,Park,Bakery
3,East York,Coffee Shop,Park,Sporting Goods Shop,Pharmacy,Burger Joint,Bank,Pizza Place,Sandwich Place,Fast Food Restaurant,Indian Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Park,Gym,Grocery Store,Liquor Store,Beer Store
5,Mississauga,Hotel,Coffee Shop,Gym / Fitness Center,Fried Chicken Joint,Sandwich Place,Burrito Place,Middle Eastern Restaurant,American Restaurant,Mediterranean Restaurant,Donut Shop
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Grocery Store,Park,Japanese Restaurant,Sandwich Place,Shopping Mall,Café
7,Queen's Park,Coffee Shop,Burger Joint,Japanese Restaurant,Gym,Diner,Yoga Studio,Italian Restaurant,Fast Food Restaurant,Nightclub,Liquor Store
8,Scarborough,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Pizza Place,Breakfast Spot,Bakery,Intersection,Motel,Park,Pharmacy
9,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Pizza Place,Bakery,Restaurant,Music Venue,Breakfast Spot,Asian Restaurant


In [48]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 2, 2, 0, 2, 3, 2, 1], dtype=int32)

In [49]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Sandwich Place,Clothing Store,Pizza Place,Café,Park,Sushi Restaurant,Dessert Shop,Gym,Burger Joint
1,Downtown Toronto,0.0,0.000773,0.000773,0.000773,0.000773,0.000773,0.001546,0.002318,0.001546,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Park,American Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Bookstore,Pub,Park,Bakery
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Park,Sporting Goods Shop,Pharmacy,Burger Joint,Bank,Pizza Place,Sandwich Place,Fast Food Restaurant,Indian Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Sandwich Place,Pharmacy,Fast Food Restaurant,Coffee Shop,Park,Gym,Grocery Store,Liquor Store,Beer Store
